<a href="https://colab.research.google.com/github/mertege/FMCW-Data-Classification-/blob/main/FMCW_Data_CNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Spectrogram'lar üzerinden CNN koşturuluyor.
# 5-fold koşturuldu ve training datasına augmentation yapıldı. 
# Fast train data'sına slow'a göre 2 kat augmentation yapıldı.
# Mean test accuracy is 0.8882352828979492, mean test f1 score is 0.8649607780428763, max test accuracy is 0.9411764740943909, max test f1 score is 0.9328063241106719, min test accuracy is 0.8235294222831726, min test f1 score is 0.7875000000000001
# 326.7857129573822

In [1]:
from keras.models import Sequential
from tensorflow.keras import layers
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Activation, Dropout, Flatten, Dense, BatchNormalization, Normalization
import tensorflow as tf
import scipy.io
import numpy as np
import cv2
import matplotlib.pyplot as plt
import random
from numpy.random import seed
from sklearn.model_selection import KFold, StratifiedKFold
import time
from sklearn.metrics import precision_recall_fscore_support

In [2]:
pip install mat73

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
import mat73
spectrograms_shuffled = mat73.loadmat('/content/drive/MyDrive/spectrograms_shuffled.mat')
spectrograms_shuffled = spectrograms_shuffled['spectrograms_shuffled']
spectrograms_shuffled = np.transpose(spectrograms_shuffled, (2, 0, 1))
spectrograms_label_shuffled = scipy.io.loadmat('/content/drive/MyDrive/spectrograms_label_shuffled.mat')
spectrograms_label_shuffled = spectrograms_label_shuffled['spectrograms_label_shuffled']  

In [5]:
# Downscale Images
spectrograms_shuffled_downscaled = np.zeros((170,50,412))
for ii in range(spectrograms_shuffled.shape[0]):
  aa = np.float32(spectrograms_shuffled[ii,:,:])
  img = cv2.cvtColor(aa, cv2.COLOR_GRAY2BGR)
  img_gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
  spectrograms_shuffled_downscaled[ii,:,:] = cv2.resize(img_gray, dsize=(412, 50), interpolation=cv2.INTER_CUBIC)


In [6]:
del spectrograms_shuffled

In [7]:
# Add new axis
print(spectrograms_shuffled_downscaled.shape[1:])
spectrograms_shuffled_downscaled = spectrograms_shuffled_downscaled[:,:,:,np.newaxis] 
print(spectrograms_shuffled_downscaled.shape[1:])

(50, 412)
(50, 412, 1)


In [10]:
t = time.time()
# ---------------- Augmente and shuffle (train and test) data data ----------------
data_augmentation = tf.keras.Sequential([
  layers.RandomFlip("horizontal_and_vertical"),
  layers.RandomRotation(0.1),
])
# ---------------- Parameters ----------------
test_accuracy_per_fold = []
f1_score_per_fold = []
repeat_of_augmentation_for_fast = 9
repeat_of_augmentation_for_slow = np.floor(repeat_of_augmentation_for_fast/2)
repeat_of_augmentation_for_slow = int(repeat_of_augmentation_for_slow)
size_of_validation = 40
num_folds = 5
kfold = StratifiedKFold(n_splits=num_folds, shuffle=True, random_state = None) # random_state = 1 ile split run'dan run'a sabit.
epoch_number = 60
batch_size = 32

for train, test in kfold.split(spectrograms_shuffled_downscaled,spectrograms_label_shuffled):   
  randomlist_for_validation_indx = test
  randomlist_for_train_indx = train
  # test data
  spectrograms_shuffled_downscaled_test = spectrograms_shuffled_downscaled[randomlist_for_validation_indx,:,:,:]
  spectrograms_label_shuffled_test = spectrograms_label_shuffled[randomlist_for_validation_indx,:]
  #train data
  spectrograms_shuffled_downscaled_train = spectrograms_shuffled_downscaled[randomlist_for_train_indx,:,:,:]
  spectrograms_label_shuffled_train = spectrograms_label_shuffled[randomlist_for_train_indx,:]
  # ---------------- Split labels to equal them during augmentation ----------------
  slow_indexes = np.where(spectrograms_label_shuffled_train == 0)[0]
  fast_indexes = np.delete(range(0, spectrograms_label_shuffled_train.shape[0]), slow_indexes)
  slow_spectrograms_train = spectrograms_shuffled_downscaled_train[slow_indexes,:,:,:]
  fast_spectrograms_train = spectrograms_shuffled_downscaled_train[fast_indexes,:,:,:]
  # ---------------- Augmente Train Data for Fast ----------------
  size_of_samples_fast = fast_spectrograms_train.shape[0]
  augmented_image_fast = np.zeros((size_of_samples_fast*repeat_of_augmentation_for_fast,fast_spectrograms_train.shape[1],fast_spectrograms_train.shape[2],1))
  spectrograms_fast_label = np.ones((size_of_samples_fast*(repeat_of_augmentation_for_fast+1),1))
  for kk in range(repeat_of_augmentation_for_fast):
    for ii in range(size_of_samples_fast):
      augmented_image_fast[size_of_samples_fast*kk+ii,:,:,:] = data_augmentation(fast_spectrograms_train[ii,:,:,:])
  augmented_image_fast = np.concatenate((augmented_image_fast,fast_spectrograms_train),axis=0)
  # ---------------- Augmente Train Data for Slow ----------------
  size_of_samples_slow = slow_spectrograms_train.shape[0]
  augmented_image_slow = np.zeros((size_of_samples_slow*repeat_of_augmentation_for_slow,slow_spectrograms_train.shape[1],slow_spectrograms_train.shape[2],1))
  spectrograms_slow_label = np.zeros((size_of_samples_slow*(repeat_of_augmentation_for_slow+1),1))
  for kk in range(repeat_of_augmentation_for_slow):
    for ii in range(size_of_samples_slow):
      augmented_image_slow[size_of_samples_slow*kk+ii,:,:,:] = data_augmentation(slow_spectrograms_train[ii,:,:,:])
  augmented_image_slow = np.concatenate((augmented_image_slow,slow_spectrograms_train),axis=0)
  # ---------------- Concat Fast and Slow ----------------
  augmented_image = np.concatenate((augmented_image_fast,augmented_image_slow),axis=0)
  spectrograms_label_shuffled_concat = np.concatenate((spectrograms_fast_label,spectrograms_slow_label),axis=0)

  # ---------------- Augmente Train Data ----------------
  # size_of_samples = spectrograms_shuffled_downscaled_train.shape[0]
  # augmented_image = np.zeros((size_of_samples*repeat_of_augmentation,spectrograms_shuffled_downscaled_train.shape[1],spectrograms_shuffled_downscaled_train.shape[2],1))
  # spectrograms_label_shuffled_concat = spectrograms_label_shuffled_train
  # for kk in range(repeat_of_augmentation):
  #   spectrograms_label_shuffled_concat = np.concatenate((spectrograms_label_shuffled_concat, spectrograms_label_shuffled_train),axis=0)
  #   for ii in range(size_of_samples):
  #     augmented_image[size_of_samples*kk+ii,:,:,:] = data_augmentation(spectrograms_shuffled_downscaled_train[ii,:,:,:])
  # augmented_image = np.concatenate((augmented_image,spectrograms_shuffled_downscaled_train),axis=0)

  layer = Normalization(axis=None)
  layer.adapt(spectrograms_shuffled_downscaled_test)
  spectrograms_shuffled_downscaled_test = layer(spectrograms_shuffled_downscaled_test)
  layer = Normalization(axis=None)
  layer.adapt(augmented_image)
  augmented_image = layer(augmented_image)

  # ---------------- Neural Network Architecture ----------------
  model = Sequential()

  model.add(Conv2D(8, (3, 3), input_shape=augmented_image.shape[1:]))
  model.add(BatchNormalization())
  model.add(Activation('relu'))
  model.add(MaxPooling2D(pool_size=(2, 2)))

  model.add(Conv2D(16, (3, 3), input_shape=augmented_image.shape[1:]))
  model.add(BatchNormalization())
  model.add(Activation('relu'))
  model.add(MaxPooling2D(pool_size=(2, 2)))

  model.add(Conv2D(32, (3, 3)))
  model.add(BatchNormalization())
  model.add(Activation('relu'))
  model.add(MaxPooling2D(pool_size=(2, 2)))


  # model.add(Conv2D(64, (3, 3)))
  # model.add(BatchNormalization())
  # model.add(Activation('relu'))
  # model.add(MaxPooling2D(pool_size=(2, 2)))

  model.add(Flatten())  
  model.add(Dense(32))
  model.add(Activation('relu'))
  model.add(Dropout(0.5))
  model.add(Dense(1))
  model.add(Activation('sigmoid'))

  print(model.summary())
  # ---------------- Compile and Fit ----------------
  model.compile(loss='binary_crossentropy',
                optimizer='rmsprop',
                metrics=['accuracy'])

  history = model.fit((augmented_image),(spectrograms_label_shuffled_concat),
                  epochs=epoch_number,
                  batch_size=batch_size,
                  shuffle = True,
                  validation_data = ((spectrograms_shuffled_downscaled_test) , (spectrograms_label_shuffled_test)))
  
  test_loss, test_accuracy  = model.evaluate((spectrograms_shuffled_downscaled_test),(spectrograms_label_shuffled_test),
                batch_size=batch_size)
  # ---------------- Get Test Results ----------------
  y_test_predicted = model.predict((spectrograms_shuffled_downscaled_test), batch_size=batch_size)
  # ----- Binarize y_test_predicted values -----
  y_test_predicted_binary = np.zeros(y_test_predicted.size)
  for ii in range(y_test_predicted.size):
    if y_test_predicted[ii] < 0.5:
      y_test_predicted_binary[ii] = 0
    else:
      y_test_predicted_binary[ii] = 1
  
  test_precision, test_recall, test_f1_score, support = precision_recall_fscore_support(spectrograms_label_shuffled_test, y_test_predicted_binary, average='macro')

  test_accuracy_per_fold.append(test_accuracy)
  f1_score_per_fold.append(test_f1_score)

print(f'Mean test accuracy is {sum(test_accuracy_per_fold)/num_folds}, mean test f1 score is {sum(f1_score_per_fold)/num_folds}, \
max test accuracy is {max(test_accuracy_per_fold)}, max test f1 score is {max(f1_score_per_fold)}, min test accuracy is {min(test_accuracy_per_fold)}, min test f1 score is {min(f1_score_per_fold)}')
elapsed = time.time() - t
print(elapsed)

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 48, 410, 8)        80        
                                                                 
 batch_normalization (BatchN  (None, 48, 410, 8)       32        
 ormalization)                                                   
                                                                 
 activation (Activation)     (None, 48, 410, 8)        0         
                                                                 
 max_pooling2d (MaxPooling2D  (None, 24, 205, 8)       0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 22, 203, 16)       1168      
                                                                 
 batch_normalization_1 (Batc  (None, 22, 203, 16)     

In [ ]:
# Test Amaçlı
model.predict(augmented_image)

In [ ]:
# Test Amaçlı
model.predict(spectrograms_shuffled_downscaled_test)